In [2]:
'''
In this exercise we will simulate real-time data analysis and feedback on a live acquisition system.
Since we don’t have a real microscope, this example is a bit contrived, but many of the principles its tests apply to the systems we use in the lab daily.

Launch MicroManager as in the previous exercise. Set the camera in “Noise” mode by opening the Device Property Browser and setting the “Mode” property of the “Camera” device to “Noise”.
Instead of the wave pattern you will now see images like this: [Img]

In a new python script, set up a timelapse acquisition with 100 time points in the DAPI channel with exposure time of 10 ms.
Extend your acquisition pipeline to inspect each image as it is acquired. If a pixel has a value of exactly 700 counts,
set neighboring pixels within a radius of 15 pixels to zero. Save this modified image in the .tif dataset instead of the original image.

A modified image may look something like this: [Img]
'''


'\nIn this exercise we will simulate real-time data analysis and feedback on a live acquisition system. \nSince we don’t have a real microscope, this example is a bit contrived, but many of the principles its tests apply to the systems we use in the lab daily.\n\nLaunch MicroManager as in the previous exercise. Set the camera in “Noise” mode by opening the Device Property Browser and setting the “Mode” property of the “Camera” device to “Noise”. \nInstead of the wave pattern you will now see images like this: [Img]\n\nIn a new python script, set up a timelapse acquisition with 100 time points in the DAPI channel with exposure time of 10 ms.\n\nExtend your acquisition pipeline to inspect each image as it is acquired. If a pixel has a value of exactly 700 counts, \nset neighboring pixels within a radius of 15 pixels to zero. Save this modified image in the .tif dataset instead of the original image.\n\nA modified image may look something like this: [Img]\n'

In [ ]:
from pycromanager import Bridge, Acquisition, multi_d_acquisition_events
import numpy as np
from skimage import draw

In [ ]:
## Connection to Micer-Manager (Check if needed)
bridge = Bridge()
core = bridge.get_core()

In [ ]:
# Set the parameters

num_time_points = 100
exposure_time_ms = 10
channels = ['DAPI']

In [ ]:
# The directory where the data will be saved
save_directory = 'C:/Users/chang/Downloads/2024_AssocRDEng_TakeHome/Exercise1'
acquisition_name = 'my_acquisition'

In [ ]:
#Generate Multi_d_acquisition events

events = multi_d_acquisition_events(
    num_time_points=num_time_points,
    time_interval_s=0,
    channels=channels,
    channel_exposures_ms=[exposure_time_ms],
    order='tpcz'
)

## Omit xy_positions

In [ ]:
# "Extend your acquisition pipeline to inspect each image as it is acquired.

with Acquisition(directory=save_directory, name= acquisition_name) as acq:
    for event in events:
        # Get current Images in each event
        image = acq.acquire_image(event)

        # Processing Images
        # Locate the position where the pixels = 700
        position700 = np.column_stack(np.where(image == 700))

        # Traverse all the pixels, and set the pixel value to 0 if it is located within a circular area with a radius of 15.
        for y, x in position700:
            rr, cc = draw.circle(y, x, 15, shape=image.shape)
            image[rr, cc] = 0  # Set those pixels as 0

        # 保存修改后的图像
        # 注意：保存逻辑需要根据实际使用的保存函数进行适配
        acq.save_image(image)


# When the context manager exits the inspect is complete and the image is saved
print(f"The image saved to {save_directory}/{acquisition_name}")